In [16]:
# Streamlit Application Code 

import streamlit as st
import cv2
import numpy as np
from skimage.feature import hog
import joblib
import os
from PIL import Image


IMG_SIZE = (128, 128) # The size you resized images to for feature extraction
MODEL_PATH = "knn_model.pkl" # Your saved model file
SCALER_PATH = "scaler.pkl"   # Your saved scaler file
TESTING_IMAGE_DIR = "testing image" 

#2. Load Model and Scaler
@st.cache_resource
def load_ml_assets():
    """Loads the trained model and scaler, caching them for Streamlit performance."""
    try:
        loaded_model = joblib.load(MODEL_PATH)
        loaded_scaler = joblib.load(SCALER_PATH)
        return loaded_model, loaded_scaler
    except FileNotFoundError:
        st.error(f"Error: Model ('{MODEL_PATH}') or Scaler ('{SCALER_PATH}') not found.")
        st.warning("Please ensure you've run 'main.ipynb' completely to train and save these files.")
        st.stop() # Stop the app if crucial files are missingtion as e:
        st.error(f"An unexpected error occurred while loading ML assets: {e}")
        st.stop() 

loaded_model, loaded_scaler = load_ml_assets()

# 3. Feature Extraction Functions 
def extract_hog_features_for_prediction(img_array):
    img_gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    img_resized = cv2.resize(img_gray, IMG_SIZE)
    features = hog(img_resized, orientations=9, pixels_per_cell=(8,8),
                   cells_per_block=(2,2), block_norm='L2-Hys', visualize=False, transform_sqrt=True)
    return features

def extract_color_features_for_prediction(img_array):
    img_resized = cv2.resize(img_array, IMG_SIZE)
    hsv = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
    hist_hsv = cv2.calcHist([hsv], [0,1,2], None, [8,8,8], [0,180,0,256,0,256])
    hist_hsv = hist_hsv.flatten()
    return hist_hsv

def get_combined_features_for_single_image(img_array):
    hog_feats = extract_hog_features_for_prediction(img_array)
    color_feats = extract_color_features_for_prediction(img_array)
    if hog_feats is None or color_feats is None:
        return None
    return np.concatenate([hog_feats, color_feats])

# 4 Prediction Function
def predict_note(image_data):
    # Convert image data to numpy array if it's from BytesIO (uploaded file)
    if isinstance(image_data, bytes):
        np_img = np.frombuffer(image_data, np.uint8)
        img_array = cv2.imdecode(np_img, cv2.IMREAD_COLOR)
    elif isinstance(image_data, np.ndarray):
        img_array = image_data
    else:
        st.error("Invalid image data format.")
        return "Error"

    if img_array is None:
        st.error("Could not decode image.")
        return "Error"

    features = get_combined_features_for_single_image(img_array)

    if features is not None:
        scaled_features = loaded_scaler.transform([features])
        prediction = loaded_model.predict(scaled_features)[0]
        
        if prediction == 0:
            return "REAL Note"
        else:
            return "COUNTERFEIT Note"
    else:
        return "Error: Could not extract features."

#  5. Streamlit App Layout
st.set_page_config(page_title="Counterfeit Money Detector")

st.title(" Counterfeit Money Detector")
st.markdown("Upload an image of a banknote or select a sample to predict if it's real or counterfeit.")

# File Uploader 
st.subheader("1. Upload Your Own Note Image")
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)
    st.write("") 

    # Convert PIL Image to OpenCV format for feature extraction
    img_cv = np.array(image)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_RGB2BGR)

    # Make prediction
    prediction_result = predict_note(img_cv)
    if "Error" not in prediction_result:
        if "REAL" in prediction_result:
            st.success(f"Prediction: This appears to be a **{prediction_result}**")
        else:
            st.error(f"Prediction: This appears to be a **{prediction_result}**")
    else:
        st.warning(prediction_result)


# Sample Image Selector 
st.subheader("2. Or, Select a Sample Image")

# Get list of sample images
sample_images = ["--- Select ---"]
if os.path.exists(TESTING_IMAGE_DIR):
    for f in os.listdir(TESTING_IMAGE_DIR):
        if f.lower().endswith(('.jpg', '.jpeg', '.png')):
            sample_images.append(f)
else:
    st.warning(f"'{TESTING_IMAGE_DIR}' folder not found. No sample images loaded.")

selected_sample = st.selectbox("Choose a sample image from the 'testing image' folder:", sample_images)

if selected_sample != "--- Select ---":
    sample_img_path = os.path.join(TESTING_IMAGE_DIR, selected_sample)
    if os.path.exists(sample_img_path):
        sample_image = Image.open(sample_img_path).convert("RGB")
        st.image(sample_image, caption=f"Selected Sample: {selected_sample}", use_column_width=True)
        st.write("")

        # Convert PIL Image to OpenCV format
        img_cv_sample = np.array(sample_image)
        img_cv_sample = cv2.cvtColor(img_cv_sample, cv2.COLOR_RGB2BGR)
        
        # Make prediction
        prediction_result_sample = predict_note(img_cv_sample)
        if "Error" not in prediction_result_sample:
            if "REAL" in prediction_result_sample:
                st.success(f"Prediction: This appears to be a **{prediction_result_sample}**")
            else:
                st.error(f"Prediction: This appears to be a **{prediction_result_sample}**")
        else:
            st.warning(prediction_result_sample)
    else:
        st.error(f"Sample image not found at: {sample_img_path}")

2025-05-25 00:18:08.723 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 00:18:08.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 00:18:08.726 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 00:18:08.727 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 00:18:08.728 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 00:18:08.729 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 00:18:08.730 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-25 00:18:08.730 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [17]:
streamlit run app.ipynb

SyntaxError: invalid syntax (659043378.py, line 1)